In [1]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Generating prior probability thresholds
# Flatten all genre lists into one long list
all_genres = sorted(set(genre for sublist in df_train["genres"] for genre in sublist))

# Flatten the genre lists
all_genres_comb = [genre for genre_list in df_train['genres'] for genre in genre_list]

# Count each genre
genre_counts = Counter(all_genres_comb)

# Total number of samples
total_samples = len(df_train)

# Compute prior probability for each genre
genre_priors = {genre: count / total_samples for genre, count in genre_counts.items()}

# Display
prior_threshold=[]
for genre, prob in sorted(genre_priors.items()):
    print(f"{genre}: {prob:.4f}")
    prior_threshold.append(prob)
prior_threshold

Action: 0.3128
Adventure: 0.2180
Comedy: 0.4091
Drama: 0.2069
Fantasy: 0.2368
Historical: 0.0743
Kids: 0.0834
Mecha: 0.0763
Music: 0.0514
Mystery: 0.0666
Romance: 0.1678
School: 0.1360
Sci-Fi: 0.1993
Seinen: 0.0712
Shoujo: 0.0565
Shounen: 0.1603
Slice of Life: 0.1410
Sports: 0.0507
Super Power: 0.0500
Supernatural: 0.1277


[0.3127808988764045,
 0.21797752808988763,
 0.40912921348314607,
 0.20688202247191012,
 0.23679775280898877,
 0.07429775280898876,
 0.08342696629213484,
 0.07626404494382022,
 0.05140449438202247,
 0.06657303370786517,
 0.1678370786516854,
 0.13595505617977527,
 0.19929775280898876,
 0.07120786516853933,
 0.056460674157303374,
 0.16025280898876404,
 0.14101123595505619,
 0.050702247191011234,
 0.05,
 0.12766853932584268]

In [3]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef

def optimize_thresholds_with_precision_constraint(probs, labels, thresholds=np.linspace(0, 1, 1000), min_precision=0.5):
    best_thresholds = []
    for i in range(probs.shape[1]):
        best_f1 = 0
        best_thresh = 0.5
        for t in thresholds:
            pred = (probs[:, i] >= t).astype(int)
            precision = precision_score(labels[:, i], pred, zero_division=0)
            f1 = f1_score(labels[:, i], pred, zero_division=0)
            if f1 > best_f1 and precision >= min_precision:
                best_f1 = f1
                best_thresh = t
        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best threshold = {best_thresh:.2f}, F1 = {best_f1:.4f}")
    return (best_thresholds)

def compute_tpr_tnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return tpr, tnr

def optimize_thresholds_with_mcc_constraint(probs, labels, thresholds=np.linspace(0, 1, 1000)):
    best_thresholds = []

    for i in range(probs.shape[1]):
        best_mcc = -1
        best_thresh = 0.5

        for t in thresholds:
            pred = (probs[:, i] >= t).astype(int)
            tpr, tnr = compute_tpr_tnr(labels[:, i], pred)
            mcc = matthews_corrcoef(labels[:, i], pred)
            if mcc > best_mcc:
                best_mcc = mcc
                best_thresh = t
 

        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best threshold = {best_thresh:.2f}, MCC = {best_mcc:.4f}")
    
    return best_thresholds

#optimised_thresh=optimize_thresholds_with_precision_constraint(val_prob,val_labels)
#optimised_thresh=optimize_thresholds_with_mcc_constraint(val_prob,val_labels)

In [45]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)

#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting


# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
#display(y)
vectorizer = TfidfVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4),min_df=2,max_df=0.8)
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

classifier = OneVsRestClassifier(MultinomialNB(alpha=0.1))#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)  

OneVsRestClassifier(estimator=MultinomialNB(alpha=0.1))

In [ ]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency:")
    
    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Combine words with their class-specific frequencies
    word_freq_pairs = list(zip(feature_names, class_counts))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_freq_pairs, key=lambda x: x[1], reverse=True)

    # Display the top 10 words with the highest frequencies
    print("Top 10 words with highest frequency:")
    for word, count in sorted_words[:20]:
        print(f"{word}: {count}")
        
    print("\n" + "=" * 50 + "\n")



Class 'Action' word frequency:
Top 10 words with highest frequency:
world: 1036
new: 607
life: 602
year: 550
power: 454
friend: 442
girl: 422
school: 413
human: 408
mysterious: 401
earth: 400
battle: 399
city: 383
war: 372
day: 362
young: 354
known: 349
time: 332
fight: 323
story: 301


Class 'Adventure' word frequency:
Top 10 words with highest frequency:
world: 824
life: 414
new: 410
friend: 367
year: 350
power: 315
earth: 289
time: 282
young: 273
girl: 271
mysterious: 271
human: 256
named: 255
day: 242
story: 240
boy: 237
way: 226
journey: 218
city: 215
people: 213


Class 'Comedy' word frequency:
Top 10 words with highest frequency:
school: 1015
girl: 915
life: 796
world: 780
new: 779
friend: 683
day: 579
time: 498
student: 464
episode: 457
high: 457
year: 449
story: 436
love: 377
high school: 375
make: 369
series: 366
way: 345
family: 325
boy: 306


Class 'Drama' word frequency:
Top 10 words with highest frequency:
life: 610
school: 497
girl: 470
world: 463
year: 422
friend: 405
n

In [38]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency and probability:")

    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class (already calculated previously)
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)

    # Combine words with their counts and probabilities
    word_stats = list(zip(feature_names, class_counts, prob[0]))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)

    # Display the top 10 words with the highest frequencies and their probabilities
    print("Top 10 words with highest frequency and probability:")
    print(f"{'Word':<20} {'Frequency':<10} {'Probability':<15}")
    print("=" * 50)
    for word, count, probability in sorted_words[:10]:
        print(f"{word:<20} {count:<10} {probability:.4f}")
        
    print("\n" + "=" * 50 + "\n")


Class 'Action' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
battle               30.714399709121178 0.0008
power                30.131660926729957 0.0013
earth                28.775194704395474 0.0012
war                  27.966394426550444 0.0008
city                 27.477736374407822 0.0013
human                27.37293760629922 0.0017
fight                25.09881665533962 0.0007
known                23.38537588058216 0.0009
series               22.483871014160222 0.0022
game                 21.88208379177676 0.0016


Class 'Adventure' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
power                21.090730966950844 0.0014
adventure            21.000099767178142 0.0007
earth                20.805674547393153 0.0013
journey              19.001178027265585 0.0005
boy                  18.891187580438608 0.001

In [ ]:
# Step 1: Input Text Preprocessing
sample_text = "pilot battle planet robot fight in space"
print(f"Sample Text: {sample_text}")

# Vectorize the input text using the same vectorizer used for training
X_sample = vectorizer.transform([sample_text])
pred=classifier.predict_proba(X_sample)
print("predicted probabilities")
for i in range(0,len(pred[0].tolist())):
    print(f"{all_genres[i]}={pred[0][i]}")
# Step 2: Mapping Features
print("\nStep 2: Mapping Features")
present_word_indices = X_sample.nonzero()[1]  # Get indices of non-zero features
feature_names = vectorizer.get_feature_names_out()

print(f"Words present in the sample:")
for index in present_word_indices:
    print(f"- {feature_names[index]} (Index: {index})")

# Step 3: Extracting Model Parameters
print("\nStep 3: Extracting Model Parameters")
for class_index, estimator in enumerate(classifier.estimators_):
    class_name = mlb.classes_[class_index]
    print(f"\nClass: {class_name}")

    # Get the prior probability for the class
    prior_log = estimator.class_log_prior_[0]
    prior = np.exp(prior_log)
    print(f"Log Prior: {prior_log:.4f}, Prior Probability: {prior:.4f}")

    # Get the log probabilities for words in the class
    log_prob = estimator.feature_log_prob_[0]

    # Step 4: Calculating Log Probabilities
    log_prob_class = prior_log  # Start with the log prior

    print("\nCalculating log probability:")
    for index in present_word_indices:
        word = feature_names[index]
        word_log_prob = log_prob[index]
        log_prob_class += word_log_prob
        print(f"Word: {word:<20} Log P(word|{class_name}): {word_log_prob:.4f}")

    # Step 5: Exponentiation to Get Final Probability
    final_prob = np.exp(log_prob_class)
    print(f"\nLog Sum for {class_name}: {log_prob_class:.4f}")
    print(f"Final Probability for {class_name}: {final_prob:.4f}")


Sample Text: pilot battle planet robot fight in space
predicted probabilities
Action=0.9782900665570801
Adventure=0.7902130161703154
Comedy=0.045823235119317275
Drama=0.11854035354265431
Fantasy=0.008760036390328992
Historical=0.00033741773093282144
Kids=0.016330815543897998
Mecha=0.9996635088045812
Music=0.0004826158807966897
Mystery=6.392538670231276e-05
Romance=0.005021570394863842
School=0.0001289826277087438
Sci-Fi=0.9999339752660713
Seinen=0.006037486045298855
Shoujo=9.257094658479882e-07
Shounen=0.17990872489299573
Slice of Life=3.388315669380318e-05
Sports=0.0008339333859334194
Super Power=0.007333325875266134
Supernatural=5.230719980664343e-05

Step 2: Mapping Features
Words present in the sample:
- battle (Index: 764)
- fight (Index: 3041)
- pilot (Index: 6445)
- planet (Index: 6481)
- robot (Index: 7307)
- space (Index: 8165)

Step 3: Extracting Model Parameters

Class: Action
Log Prior: -0.3751, Prior Probability: 0.6872

Calculating log probability:
Word: battle           

In [26]:
# List of specific words to check
specific_words = ["love", "action", "friend", "school", "battle", "fight"]  # Modify this list as needed

# Classes to compare (you can customize this)
class1 = "Action"
class2 = "Romance"

# Find the indices of the classes
try:
    class1_index = list(mlb.classes_).index(class1)
    class2_index = list(mlb.classes_).index(class2)
except ValueError:
    print(f"One or both classes '{class1}' and '{class2}' not found.")
    exit()

# Print the header
print(f"{'Word':<15} {'Class1':<10} {'Class2':<10} {'Log Odds Ratio':<20}")
print("=" * 60)

# Loop through each specific word
for word in specific_words:
    # Find the index of the word in the vocabulary
    try:
        word_index = vectorizer.vocabulary_[word]
    except KeyError:
        print(f"{word:<15} Not in vocab")
        continue

    # Get the estimators for both classes
    estimator1 = classifier.estimators_[class1_index]
    estimator2 = classifier.estimators_[class2_index]

    # Get log probabilities for the word in both classes
    log_prob1 = estimator1.feature_log_prob_[0, word_index]
    log_prob2 = estimator2.feature_log_prob_[0, word_index]

    # Calculate log odds ratio
    log_odds_ratio = log_prob1 - log_prob2

    # Print the results
    print(f"{word:<15} {log_prob1:.4f}    {log_prob2:.4f}    {log_odds_ratio:.4f}")

print("=" * 60)


Word            Class1     Class2     Log Odds Ratio      
love            -6.2083    -6.9367    0.7284
action          -8.0055    -7.6971    -0.3084
friend          -5.9538    -6.1105    0.1567
school          -5.6020    -5.9294    0.3274
battle          -7.1918    -6.5452    -0.6466
fight           -7.2422    -6.7200    -0.5222


In [31]:
# List of specific words to check
specific_words = ["love", "action", "friend", "school", "battle"]  # Modify this list as needed

# Loop through each class within the OneVsRestClassifier
for word in specific_words:
    print(f"Word: '{word}'")
    print(f"{'Class':<20} {'Probability':<15}")
    print("=" * 50)

    # Find the index of the word in the vocabulary
    try:
        word_index = vectorizer.vocabulary_[word]
    except KeyError:
        print(f"Word '{word}' not found in vocabulary.")
        print("=" * 50 + "\n")
        continue

    # Loop through each class and print the word probability
    for class_index, estimator in enumerate(classifier.estimators_):
        # Get the class name
        class_name = mlb.classes_[class_index]

        # Extract the log probabilities from the Naive Bayes model
        log_prob = estimator.feature_log_prob_

        # Convert log probabilities to normal probabilities
        prob = np.exp(log_prob)

        # Get the probability of the word given the class
        word_prob = prob[0, word_index]
        word_log_prob=log_prob[0, word_index]

        # Print the class name and word probability
        print(f"{class_name:<20} {word_prob:.4f}")

    print("=" * 50 + "\n")


Word: 'love'
Class                Probability    
Word 'love' not found in vocabulary.

Word: 'action'
Class                Probability    
Action               0.0001
Adventure            0.0001
Comedy               0.0001
Drama                0.0001
Fantasy              0.0001
Historical           0.0001
Kids                 0.0001
Mecha                0.0001
Music                0.0001
Mystery              0.0001
Romance              0.0001
School               0.0001
Sci-Fi               0.0001
Seinen               0.0001
Shoujo               0.0001
Shounen              0.0001
Slice of Life        0.0001
Sports               0.0001
Super Power          0.0001
Supernatural         0.0001

Word: 'friend'
Class                Probability    
Word 'friend' not found in vocabulary.

Word: 'school'
Class                Probability    
Word 'school' not found in vocabulary.

Word: 'battle'
Class                Probability    
Word 'battle' not found in vocabulary.



In [39]:
import pandas as pd
import numpy as np

# Create an empty dictionary to store genres as keys and top words with probabilities as lists
genre_words = {}

# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    # Get the genre name
    genre_name = mlb.classes_[class_index]
    
    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()
    
    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_
    
    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)
    
    # Combine words with their probabilities
    word_stats = list(zip(feature_names, prob[0]))
    
    # Sort words by probability in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)[:5]
    
    # Create a list to store the top 5 words and their probabilities as strings
    top_words_with_probs = [f"{word} ({probability:.4f})" for word, probability in sorted_words]
    
    # Store the top words with probabilities in the dictionary with genre as the key
    genre_words[genre_name] = top_words_with_probs

# Convert the dictionary to a DataFrame
df_top_words = pd.DataFrame.from_dict(genre_words, orient='index', columns=[f'Top Word {i+1} (Prob)' for i in range(5)])

# Display the DataFrame
display(df_top_words)


,Top Word 1 (Prob),Top Word 2 (Prob),Top Word 3 (Prob),Top Word 4 (Prob),Top Word 5 (Prob)
Action,episode (0.0028),series (0.0022),anime (0.0021),love (0.0021),boy (0.0020)
Adventure,episode (0.0025),anime (0.0021),series (0.0021),love (0.0019),high school (0.0018)
Comedy,human (0.0021),boy (0.0019),power (0.0018),city (0.0018),earth (0.0018)
Drama,episode (0.0026),series (0.0022),anime (0.0021),character (0.0018),game (0.0018)
Fantasy,episode (0.0027),series (0.0022),anime (0.0020),boy (0.0019),high school (0.0018)
Historical,episode (0.0024),series (0.0021),human (0.0019),anime (0.0018),boy (0.0018)
Kids,episode (0.0023),series (0.0020),human (0.0018),boy (0.0017),love (0.0017)
Mecha,episode (0.0024),series (0.0020),anime (0.0019),boy (0.0019),human (0.0017)
Music,episode (0.0024),series (0.0021),human (0.0019),boy (0.0018),anime (0.0018)
Mystery,episode (0.0025),series (0.0021),anime (0.0019),human (0.0019),boy (0.0018)


In [20]:
import pandas as pd
import numpy as np

# Create an empty dictionary to store genre and top words
genre_words = {"Genre": [], "Top Word 1": [], "Top Word 2": [], "Top Word 3": [], "Top Word 4": [], "Top Word 5": []}

# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    # Get the genre name
    genre_name = mlb.classes_[class_index]
    
    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()
    
    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_
    
    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)
    
    # Combine words with their probabilities
    word_stats = list(zip(feature_names, prob[0]))
    
    # Sort words by probability in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)[:5]
    
    # Extract the top 5 words for this genre
    top_words = [word for word, _ in sorted_words]
    
    # Store in the dictionary
    genre_words["Genre"].append(genre_name)
    for i in range(5):
        genre_words[f"Top Word {i+1}"].append(top_words[i] if i < len(top_words) else '')

# Convert to DataFrame
df_top_words = pd.DataFrame(genre_words)

# Display the DataFrame
display(df_top_words)
df_top_words.to_csv("words TFDIFMNB.csv")


,Genre,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5
0,Action,girl,school,life,story,episode
1,Adventure,school,girl,life,story,new
2,Comedy,world,life,girl,story,school
3,Drama,world,girl,school,new,life
4,Fantasy,school,girl,life,new,story
5,Historical,girl,school,world,life,new
6,Kids,girl,school,world,life,new
7,Mecha,girl,school,world,life,story
8,Music,world,girl,school,life,new
9,Mystery,girl,world,school,life,new


In [19]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency and probability:")

    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class (already calculated previously)
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)

    # Combine words with their counts and probabilities
    word_stats = list(zip(feature_names, class_counts, prob[0]))

    
    sorted_words = sorted(word_stats, key=lambda x: x[2], reverse=True)

    # Display the top 10 words with the highest frequencies and their probabilities
    print("Top 10 words with highest frequency and probability:")
    print(f"{'Word':<20} {'Frequency':<10} {'Probability':<15}")
    print("=" * 50)
    for word, count, probability in sorted_words[0:5]:
        print(f"{word:<20} {count:<10} {probability:.4f}")
        
    print("\n" + "=" * 50 + "\n")


Class 'Action' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
girl                 23.20604349156397 0.0038
school               22.22172762061093 0.0037
life                 30.518717822260026 0.0032
story                22.43560423819046 0.0030
episode              18.16317049678781 0.0027


Class 'Adventure' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
school               7.021763915518274 0.0037
girl                 15.527458285555731 0.0035
life                 21.23209520185186 0.0031
story                18.086930303661905 0.0027
new                  23.361935277302965 0.0027


Class 'Comedy' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
world                42.905534166486014 0.0034
life                 45.99856300383052

In [25]:
import pandas as pd
import numpy as np

# Initialize the DataFrame with column names
influential_words_df = pd.DataFrame(columns=[f"Top Word {i+1}" for i in range(5)])

for class_index, estimator in enumerate(classifier.estimators_):
    genre_name = mlb.classes_[class_index]
    
    # Get feature names and log probabilities
    feature_names = vectorizer.get_feature_names_out()
    log_prob = estimator.feature_log_prob_

    # Calculate log odds (difference between current class and the average of others)
    log_odds = log_prob[0] - np.mean(log_prob, axis=0)

    # Combine words with their log-odds
    word_stats = list(zip(feature_names, log_odds))

    # Sort words by absolute log-odds in descending order
    sorted_words = sorted(word_stats, key=lambda x: abs(x[1]), reverse=True)

    # Select the top 5 most influential words
    top_5_words = [f"{word} ({log_odds:.4f}) " for word, log_odds in sorted_words[:5]]#({log_odds:.4f})

    # Add the top words to the DataFrame as a new row
    influential_words_df.loc[genre_name] = pd.Series(top_5_words, index=influential_words_df.columns)

# Display the DataFrame
display(influential_words_df)

#influential_words_df.to_csv('MNB words.csv',  index=False)


,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5
Action,gokuu (-2.2827),gunpla (-2.0762),kirito (-2.0265),kamui (-2.0214),educational (1.9891)
Adventure,pokémon (-2.3152),sinbad (-2.3018),gohan (-2.2846),eureka (-2.2389),goemon (-2.2051)
Comedy,comedic (-2.1100),doraemon (-2.1016),tenchi (-2.1006),hilarious (-2.0666),arale (-2.0144)
Drama,karasuno (-2.2872),taisei (-2.0778),sensou (-2.0447),hiroki (-2.0149),arslan (-2.0026)
Fantasy,pokémon (-2.5996),precure (-2.5336),pokemon (-2.2959),kirito (-2.2385),rimuru (-2.2250)
Historical,hijikata (-2.5632),genji (-2.5353),melos (-2.4556),lucius (-2.4449),holo (-2.4378)
Kids,pokémon (-3.2473),pokemon (-2.9245),backkom (-2.8969),shobu (-2.8557),pikachu (-2.8521)
Mecha,zeon (-3.0724),mobile suit (-2.9506),gundam (-2.9076),gunpla (-2.8360),universal century (-2.5962)
Music,shower tv (-2.8074),space shower (-2.8074),space shower tv (-2.8074),pripara (-2.7673),id directed (-2.7288)
Mystery,mikiya (-2.8941),edogawa (-2.8809),conan edogawa (-2.7702),kindaichi (-2.7386),detective boy (-2.6883)


In [46]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, hamming_loss, jaccard_score, roc_auc_score
import numpy as np
import pandas as pd

def predict_genres_with_naives(classifier, vectorizer, df_source, mlb, id2label, threshold=0.5):
    # Vectorize clean synopsis
    X = vectorizer.transform(df_source['clean_synopsis'])
    y_true = mlb.transform(df_source['genres'])

    # Predict probabilities and apply threshold
    probabilities = classifier.predict_proba(X)
    probabilities = np.array(probabilities)  # list of arrays → array
    if isinstance(probabilities, list):  # OneVsRestClassifier returns list of arrays
        probabilities = np.vstack([p[:, 1] for p in probabilities]).T
    predictions = (probabilities >= threshold).astype(int)

    # Decode predicted genres using id2label
    predicted_genres = []
    for row in predictions:
        genres = [id2label[i] for i, val in enumerate(row) if val == 1]
        predicted_genres.append(genres)

    # === Calculate metrics ===
    f1 = f1_score(y_true, predictions, average='macro')
    precision = precision_score(y_true, predictions, average='macro')
    recall = recall_score(y_true, predictions, average='macro')
    accuracy = accuracy_score(y_true, predictions)
    hamming = hamming_loss(y_true, predictions)
    jaccard = jaccard_score(y_true, predictions, average='macro')
    hit_rate = (np.logical_and(y_true, predictions).sum(axis=1) > 0).mean()
    try:
        roc_auc = roc_auc_score(y_true, probabilities, average='macro')
    except ValueError:
        roc_auc = np.nan

    metrics_df = pd.DataFrame({
        'Metric': [
            'F1 Score', 'Precision', 'Recall', 'Exact Match Accuracy',
            'Hamming Loss', 'Jaccard Score', 'Hit Rate', 'ROC AUC'
        ],
        'Value': [
            f1, precision, recall, accuracy,
            hamming, jaccard, hit_rate, roc_auc
        ]
    })

    # Build result DataFrame
    result_df = pd.DataFrame({
        "synopsis": df_source["synopsis"].values,
        "true_genres": df_source["genres"].values,
        "predicted_genres": predicted_genres
    })

    print("\n📊 Evaluation Metrics:")
    print(metrics_df)

    return result_df, metrics_df, probabilities, y_true, predictions

print("0.5 Thresh")
use_thresh=0.5
id2label = {i: label for i, label in enumerate(mlb.classes_)}
#df_train_results, train_metrics, train_prob, train_labels, train_pred  = predict_genres_with_naives(classifier, vectorizer, df_train, mlb, id2label, threshold=use_thresh)
df_val_results, val_metrics, val_prob, val_labels, val_pred = predict_genres_with_naives(classifier, vectorizer, df_val, mlb, id2label, threshold=use_thresh)
df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)


# print("prior Thresh")
# use_thresh=prior_threshold
# df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

print("F1 Thresh")
optimised_thresh=optimize_thresholds_with_precision_constraint(val_prob,val_labels)
use_thresh=optimised_thresh
df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

# print("MCC Thresh")
# optimised_thresh=optimize_thresholds_with_mcc_constraint(val_prob,val_labels)
# use_thresh=optimised_thresh
# df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

0.5 Thresh

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.394592
1             Precision  0.833870
2                Recall  0.290355
3  Exact Match Accuracy  0.105381
4          Hamming Loss  0.112164
5         Jaccard Score  0.260417
6              Hit Rate  0.633408
7               ROC AUC  0.847346

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.383092
1             Precision  0.805268
2                Recall  0.278333
3  Exact Match Accuracy  0.093154
4          Hamming Loss  0.112795
5         Jaccard Score  0.249858
6              Hit Rate  0.598204
7               ROC AUC  0.848243
F1 Thresh
Label 0: Best threshold = 0.37, F1 = 0.6957
Label 1: Best threshold = 0.28, F1 = 0.6050
Label 2: Best threshold = 0.37, F1 = 0.6675
Label 3: Best threshold = 0.28, F1 = 0.5101
Label 4: Best threshold = 0.37, F1 = 0.6056
Label 5: Best threshold = 0.12, F1 = 0.6154
Label 6: Best threshold = 0.14, F1 = 0.5578
Label 7: Best

In [48]:
def predict_new_synopses_naives(classifier, vectorizer, new_synopses, id2label, threshold=0.5):
    """
    Predict genres for a list of new synopses using the trained Naive Bayes model.
    """
    # Vectorize the input synopses
    X = vectorizer.transform(new_synopses)

    # Predict probabilities and apply threshold
    probabilities = classifier.predict_proba(X)
    probabilities = np.array(probabilities)  # list of arrays → array
    if isinstance(probabilities, list):  # OneVsRestClassifier returns list of arrays
        probabilities = np.vstack([p[:, 1] for p in probabilities]).T
    predictions = (probabilities >= threshold).astype(int)

    # Decode predicted genres using id2label
    predicted_genres = []
    for row in predictions:
        genres = [id2label[i] for i, val in enumerate(row) if val == 1]
        predicted_genres.append(genres)

    # Build result DataFrame
    result_df = pd.DataFrame({
        "synopsis": new_synopses,
        "predicted_genres": predicted_genres
    })

    print("\n🎬 Predicted Genres for New Synopses:")
    print(result_df)

    return result_df

# Example usage
new_synopses = [
    "A young boy discovers his hidden powers and joins a group of heroes to fight evil forces.",
    "A romantic comedy where two strangers meet at a cafe and end up falling in love despite their differences.",
    "Love girl school romance"
]

# Predict genres for the new synopses
predicted_df = predict_new_synopses_naives(classifier, vectorizer, new_synopses, id2label, threshold=0.5)
display(predicted_df)



🎬 Predicted Genres for New Synopses:
                                            synopsis  \
0  A young boy discovers his hidden powers and jo...   
1  A romantic comedy where two strangers meet at ...   
2                           Love girl school romance   

                   predicted_genres  
0                       [Adventure]  
1  [Comedy, Romance, Slice of Life]  
2                 [Romance, School]  


,synopsis,predicted_genres
0,A young boy discovers his hidden powers and jo...,[Adventure]
1,A romantic comedy where two strangers meet at ...,"[Comedy, Romance, Slice of Life]"
2,Love girl school romance,"[Romance, School]"


In [9]:
print("F1 Thresh")
optimised_thresh=optimize_thresholds_with_precision_constraint(val_prob,val_labels)
use_thresh=optimised_thresh
df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)

F1 Thresh
Label 0: Best threshold = 0.37, F1 = 0.6957
Label 1: Best threshold = 0.28, F1 = 0.6050
Label 2: Best threshold = 0.37, F1 = 0.6675
Label 3: Best threshold = 0.28, F1 = 0.5101
Label 4: Best threshold = 0.37, F1 = 0.6056
Label 5: Best threshold = 0.12, F1 = 0.6154
Label 6: Best threshold = 0.14, F1 = 0.5578
Label 7: Best threshold = 0.11, F1 = 0.6057
Label 8: Best threshold = 0.13, F1 = 0.4706
Label 9: Best threshold = 0.14, F1 = 0.5253
Label 10: Best threshold = 0.23, F1 = 0.6062
Label 11: Best threshold = 0.41, F1 = 0.6296
Label 12: Best threshold = 0.29, F1 = 0.6721
Label 13: Best threshold = 0.17, F1 = 0.2727
Label 14: Best threshold = 0.16, F1 = 0.3864
Label 15: Best threshold = 0.22, F1 = 0.4651
Label 16: Best threshold = 0.27, F1 = 0.5349
Label 17: Best threshold = 0.17, F1 = 0.8049
Label 18: Best threshold = 0.12, F1 = 0.2857
Label 19: Best threshold = 0.23, F1 = 0.5962

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.531597
1        

In [ ]:
use_thresh=0.5
id2label = {i: label for i, label in enumerate(mlb.classes_)}
#df_train_results, train_metrics, train_prob, train_labels, train_pred  = predict_genres_with_naives(classifier, vectorizer, df_train, mlb, id2label, threshold=use_thresh)
df_val_results, val_metrics, val_prob, val_labels, val_pred = predict_genres_with_naives(classifier, vectorizer, df_val, mlb, id2label, threshold=use_thresh)
df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)


📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.540043
1             Precision  0.554933
2                Recall  0.540256
3  Exact Match Accuracy  0.098655
4          Hamming Loss  0.123038
5         Jaccard Score  0.379643
6              Hit Rate  0.848655
7               ROC AUC  0.821518

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.529547
1             Precision  0.549325
2                Recall  0.530849
3  Exact Match Accuracy  0.075196
4          Hamming Loss  0.127778
5         Jaccard Score  0.367733
6              Hit Rate  0.835017
7               ROC AUC  0.825164


In [ ]:

from sklearn.metrics import classification_report

print(classification_report(train_labels, train_pred, target_names=all_genres))
print(classification_report(val_labels, val_pred, target_names=all_genres))
print(classification_report(test_labels, test_pred, target_names=all_genres))

               precision    recall  f1-score   support

       Action       0.77      0.82      0.79      2227
    Adventure       0.71      0.89      0.79      1552
       Comedy       0.81      0.88      0.84      2913
        Drama       0.77      0.87      0.82      1473
      Fantasy       0.83      0.82      0.83      1686
   Historical       0.67      0.97      0.79       529
         Kids       0.66      0.98      0.79       594
        Mecha       0.63      0.95      0.75       543
        Music       0.42      0.99      0.59       366
      Mystery       0.72      0.97      0.83       474
      Romance       0.72      0.85      0.78      1195
       School       0.63      0.93      0.75       968
       Sci-Fi       0.87      0.79      0.83      1419
       Seinen       0.97      0.81      0.88       507
       Shoujo       0.95      0.87      0.90       402
      Shounen       0.77      0.90      0.83      1141
Slice of Life       0.87      0.76      0.81      1004
       Sp

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
#display(df_train_results)
# display(df_val_results)
display(df_test_results)
df_test_results.to_csv("Predict TFIDF MNB.csv")

,synopsis,true_genres,predicted_genres
0,The daily life of the Motsumoto family. The th...,"[Comedy, Slice of Life]","[Comedy, Romance, Slice of Life]"
1,"The Soul Tree, the great source of our race. ...","[Action, Adventure, Kids]","[Action, Adventure, Fantasy]"
2,"Fairies living in a fluffy forest, where both ...","[Fantasy, Kids]","[Comedy, Fantasy, Kids, Shoujo, Slice of Life]"
3,A cyborg warrior from an ancient Antarctic kin...,"[Action, Mecha, Sci-Fi]","[Action, Adventure, Mecha, Sci-Fi, Shounen]"
4,The 2018 LINE sticker set Poccolies is inspiri...,"[Kids, Slice of Life]","[Comedy, Slice of Life]"
...,...,...,...
886,A series of four animated prequels to Street F...,[Action],"[Action, Drama, Mecha, Shounen]"
887,"According to HMV, the DVD & Blu-ray volume 7 o...","[Comedy, Seinen, Sports]",[Comedy]
888,"Lucius ends up in a space station, where he ex...","[Comedy, Seinen]","[Comedy, Historical, Sci-Fi, Seinen]"
889,"The ""Megumi and Taiyou"" commercial depicts the...","[Romance, Slice of Life]",[Romance]


In [22]:
roc_auc_per_genre = roc_auc_score(test_labels, test_prob, average=None)
for label, auc in zip(all_genres, roc_auc_per_genre):
    print(f"{label}: ROC AUC = {auc:.4f}")


Action: ROC AUC = 0.8501
Adventure: ROC AUC = 0.8284
Comedy: ROC AUC = 0.7777
Drama: ROC AUC = 0.7570
Fantasy: ROC AUC = 0.8694
Historical: ROC AUC = 0.9078
Kids: ROC AUC = 0.8992
Mecha: ROC AUC = 0.9172
Music: ROC AUC = 0.8983
Mystery: ROC AUC = 0.8603
Romance: ROC AUC = 0.8250
School: ROC AUC = 0.8688
Sci-Fi: ROC AUC = 0.8958
Seinen: ROC AUC = 0.7364
Shoujo: ROC AUC = 0.8242
Shounen: ROC AUC = 0.7312
Slice of Life: ROC AUC = 0.8651
Sports: ROC AUC = 0.9705
Super Power: ROC AUC = 0.8221
Supernatural: ROC AUC = 0.8602


In [23]:
df=pd.DataFrame(roc_auc_per_genre)
display(df)
df.to_csv('TFIDFMNB.csv', float_format='%.2f', index=False)


,0
0,0.850107
1,0.828417
2,0.777686
3,0.757034
4,0.869446
5,0.907769
6,0.899170
7,0.917206
8,0.898251
9,0.860312


In [ ]:
from sklearn.metrics import matthews_corrcoef
import numpy as np
import pandas as pd

def mcc_per_label(y_true, y_pred, label_names=None):
    n_labels = y_true.shape[1]
    mcc_scores = []

    for i in range(n_labels):
        mcc = matthews_corrcoef(y_true[:, i], y_pred[:, i])
        mcc_scores.append(mcc)

    if label_names is None:
        label_names = [f"Label_{i}" for i in range(n_labels)]

    return pd.Series(mcc_scores, index=label_names, name="MCC")

MCC_labels=mcc_per_label(val_labels,val_pred,label_names=all_genres)
display(MCC_labels)

In [ ]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)

In [ ]:
#Instead of using a fixed threshold we compare it to the prior threshold from the train set
def predict_using_prior_threshold(X, classifier, priors):
    """
    Predicts genres using P(c|x) > P(c) as threshold logic.
    - classifier: trained OneVsRestClassifier
    - priors: array of prior probabilities (i.e., class proportions in y_train)
    """
    probs = classifier.predict_proba(X)
    # compare each prob with its class's prior
    preds = (probs > priors).astype(int)
    return preds





In [ ]:
#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting
from sklearn.metrics import hamming_loss

# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
priors = y_train.mean(axis=0)
#display(y)
vectorizer = TfidfVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4))
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

classifier = OneVsRestClassifier(MultinomialNB(alpha=0.1))#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)                                    #At least when it is not balanced it can overfit the train class

# from sklearn.metrics import classification_report
#samples	How well did we predict each movie's genres?	✅ Best for your task
#micro	    Overall genre prediction accuracy	            ✅ Good for global view
#macro	    Equal focus on rare and frequent genres	        ⚠️ Careful with imbalance
#weighted	Genre performance balanced by frequency	        ✅ Fair but skew-sensitive
#USE SAMPLES


print("Report for Train dataset")
#threshold=0.08
#y_pred_train=LR_OVA_prob(X_train,threshold,classifier)
y_pred_train = predict_using_prior_threshold(X_train, classifier, priors)
print_report(y_train,y_pred_train,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_train, y_pred_train, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_train, y_pred_train)
print("Hit Rate:", hr)
hl = hamming_loss(y_train, y_pred_train)
print("Hamming Loss:", hl)

print("----------------------")
print("Report for Val dataset")
#y_pred_val=LR_OVA_prob(X_val,threshold,classifier)
y_pred_val = predict_using_prior_threshold(X_val, classifier, priors)
print_report(y_val,y_pred_val,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_val, y_pred_val, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)
hl = hamming_loss(y_val, y_pred_val)
print("Hamming Loss:", hl)


print("----------------------")
print("Report for Test dataset")
#y_pred_test=LR_OVA_prob(X_test,threshold,classifier)
y_pred_test = predict_using_prior_threshold(X_test, classifier, priors)
print_report(y_test,y_pred_test,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_test, y_pred_test, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)
hl = hamming_loss(y_test, y_pred_test)
print("Hamming Loss:", hl)



display(result_table(y_train,y_pred_train,df_train,mlb))
display(result_table(y_val,y_pred_val,df_val,mlb))
display(result_table(y_test,y_pred_test,df_test,mlb))

NameError: name 'MultiLabelBinarizer' is not defined